# Freeze me
## Gruppenmitglieder

Alwin Bossert
Sören Rempel

### Imports

In [ ]:
import cv2 as cv
import numpy as np
import urllib.request

In [ ]:
# Helper functions

def get_videostream_from_file(filename):
    return cv.VideoCapture(filename)


def get_videostream_from_url(url, filename='video.mp4'):
    urllib.request.urlretrieve(url, filename)
    return cv.VideoCapture(filename)


def background_subtraction(stream, noise_reduction=False, limiter=0):
    ret, frame = stream.read()
    images = []
    fgbg = cv.createBackgroundSubtractorKNN()
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
    limiter_count = 1
    while ret:
        if limiter_count <= limiter:
            limiter_count += 1
            ret, frame = stream.read()
            continue
        limiter_count = 1
        fgmask = fgbg.apply(frame)
        if noise_reduction:
            fgmask = cv.morphologyEx(fgmask, cv.MORPH_OPEN, kernel)
        testcolor = cv.bitwise_and(frame, cv.cvtColor(fgmask, cv.COLOR_GRAY2RGB))
        masked = np.ma.masked_equal(testcolor, 0)
        images.append(masked)
        ret, frame = stream.read()
    return images


def optical_flow(stream, int_threshold=150, noise_reduction=False, limiter=0):
    ret, frame1 = stream.read()
    prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[..., 1] = 255
    images = []
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
    limiter_count = 1
    while True:
        ret, frame2 = stream.read()
        if not ret:
            break
        if limiter_count <= limiter:
            limiter_count += 1
            continue
        limiter_count = 1
        next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
        flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
        intensity = hsv[:, :, 2]
        intensity_mask = (np.logical_not(intensity < int_threshold) * 255).astype(np.uint8)
        mask = np.dstack((intensity_mask, intensity_mask, intensity_mask))
        if noise_reduction:
            mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
        masked_img = cv.bitwise_and(frame2, mask)
        masked = np.ma.masked_equal(masked_img, 0)
        images.append(masked)
        prvs = next
    return images


def average_images(images, opacity):
    avg_img = np.ma.average(images, axis=0)
    avg_img = avg_img.astype(np.uint8)
    avg_img_bgra = cv.cvtColor(avg_img, cv.COLOR_BGR2BGRA)
    alpha_channel = avg_img_bgra[:, :, 3]
    alpha_channel[np.all(avg_img_bgra[:, :, 0:3] == (0, 0, 0), 2)] = 0
    avg_img_bgra = avg_img_bgra.astype(np.float64)
    avg_img_bgra[:, :, 3] *= opacity
    avg_img_bgra = avg_img_bgra.astype(np.uint8)
    return avg_img_bgra


def video_to_freeze_picture(mode, blur_motion=False, opacity=0.5, noise_reduction=False, file=None, url=None, limit=None):
    if file:
        stream = get_videostream_from_file(file)
    elif url:
        stream = get_videostream_from_url(url)
    else:
        print("No input file provided")
        quit()
    max_frames = stream.get(cv.CAP_PROP_FRAME_COUNT)
    print("maxframes")
    print(max_frames)
    if limit is None:
        limit_val = max_frames + 3
    else:
        limit_val = int(max_frames/limit)
        print(limit_val)
    if mode.lower() == 'bgs':
        images = background_subtraction(stream, noise_reduction=noise_reduction, limiter=limit_val)
    elif mode.lower() == 'of':
        images = optical_flow(stream, noise_reduction=noise_reduction, limiter=limit_val)
    elif mode.lower() == 'both':
        print("Not implemented yet")
        quit()
    else:
        print("No valid mode provided")
        quit()
    print(len(images))
    avg_image = average_images(images, opacity=opacity)
    if blur_motion:
        avg_image = cv.blur(avg_image, (3, 3))
    stream.set(cv.CAP_PROP_POS_FRAMES, 0)
    ret, img = stream.read()
    avg_img_to_gray = cv.cvtColor(avg_image, cv.COLOR_BGR2GRAY)
    motion_mask_inv = (np.logical_not(avg_img_to_gray > 1) * 255).astype(np.uint8)
    firstframe_masked = cv.bitwise_and(img, img, mask=motion_mask_inv)
    final_img = cv.add(avg_image, cv.cvtColor(firstframe_masked, cv.COLOR_BGR2BGRA))
    return final_img

